In [ ]:
#config.py
# Конфигурация подключений
MYSQL_CONFIG = {
    'host': 'ich-edit.edu.itcareerhub.de',
    'user': 'ich1',
    'password': 'ich1_password_ilovedbs',
    'database': 'library_test'
}

MONGO_CONFIG = {
    'uri': "mongodb://ich_editor:verystrongpassword"
          "@mongo.itcareerhub.de/?readPreference=primary"
          "&ssl=false&authMechanism=DEFAULT&authSource=ich_edit",
    'database': 'ich_edit',
    'collection': 'library_110425_logs_searches'
}


In [ ]:
# search.py
import pymysql
from config import MYSQL_CONFIG

def search_books_by_author(author_name):
    # Подключение к серверу MySQL
    with pymysql.connect(**MYSQL_CONFIG) as connection:
        with connection.cursor() as cursor:
            # SQL-запрос для поиска книг по имени автора
            query = """
                SELECT id, title, author, price
                FROM books
                WHERE author LIKE %s
            """
            cursor.execute(query, ('%' + author_name + '%',)) # LIKE '%leo%'

            # Вывод результатов
            count = 0
            print("\nПодходящие книги:")
            for row in cursor:
                print(f"ID: {row[0]}, Title: {row[1]}, Author: {row[2]}, Price: {row[3]}")
                count += 1
            if count == 0:
                print("не найдены.")

# Вставляем все остальные функции

In [ ]:
# log_writer.py
from config import MONGO_CONFIG
from pymongo import MongoClient


def log_search_query(query_type, query_str):
    # Настройка подключения к MongoDB
    mongo_client = MongoClient(MONGO_CONFIG['uri'])
    searches_collection = mongo_client[MONGO_CONFIG['database']]['collection']
    searches_collection.insert_one({"query_type": query_type, "query_str": query_str})
    




In [ ]:
#log_reader.py
from config import MONGO_CONFIG
from collections import Counter

def show_popular_queries():
    mongo_client = MongoClient(MONGO_CONFIG['uri'])
    searches_collection = mongo_client[MONGO_CONFIG['database']]['collection']    
    queries = [doc["query_type"] for doc in searches_collection.find() if doc.get("query_type")]
    counter = Counter(queries)
    top = counter.most_common(5)

    if not top: #если пустой
        print("No search data found.")
        return

    print("Most frequent search queries:")
    for i, (query, count) in enumerate(top):
        print(f"{i+1}. {query} — {count} times")

In [ ]:
# main_menu.py
from search import *
from log_writer import *

def main_menu():
    while True:
        print("\nГлавное меню:")
        print("1. Поиск информации о книгах")
        print("2. Поиск информации о клиентах")
        print("3. Вывод топ-5 популярных запросов")
        print("0. Выход")

        choice = input("Выберите пункт меню (1, 2, 3 или 0): ")

        if choice == '1':
            while True:
                print("\nМеню информации о книгах:")
                print("1. Поиск книг по автору")
                print("2. Поиск книг по части названия")
                print("3. Общий доход, сгенерированный каждой книгой")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2, 3 или 0): ")

                if sub_choice == '1':
                    author_name = input("Введите имя автора (или его часть): ")
                    search_books_by_author(author_name)
                    log_type = 'search_books_by_author' # log_type = search.search_books_by_author.__name__
                    log_str = author_name
                    log_search_query(log_type, log_str)
                    
                elif sub_choice == '2':
                    title_part = input("Введите часть названия книги: ")
                    search_books_by_title(title_part)
                    log_type = 'search_books_by_title'
                    log_str = title_part
                    log_search_query(log_type, log_str)
                elif sub_choice == '3':
                    # нужна доработка
                    calculate_total_income_by_book()
                    # нужна доработка
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2, 3 или 0.")

        elif choice == '2':
            while True:
                print("\nМеню информации о клиентах:")
                print("1. Поиск книг клиента")
                print("2. Суммарные затраты каждого клиента")
                print("0. Назад")

                sub_choice = input("Выберите пункт подменю (1, 2 или 0): ")

                if sub_choice == '1':
                    search_books_by_client()
                elif sub_choice == '2':
                    calculate_total_expenses_by_user()
                elif sub_choice == '0':
                    break
                else:
                    print("Некорректный выбор. Пожалуйста, выберите 1, 2 или 0.")

        elif choice == '3':
            show_popular_queries()

        elif choice == '0':
            print("Выход из программы.")
            break
        else:
            print("Некорректный выбор. Пожалуйста, выберите 1, 2, 3 или 0.")


In [ ]:
# основная программа

from main_menu import *

main_menu()